# Manual QC of physiological features

This notebook contains the instructions (this section) and the code (next section) to perform manual QC on the Skin Conductance (SC) signal.

## Data overview

### Signal acquisition
The skin conductance was recorded on the right foot using LEAD108C with EL509 electrodes with isotonic gel, and the EDA100C-MRI amplifier (DC; constant voltage at 0.5V). 

### Preprocessing
A bidirectional lowpass filter (cutoff: 3Hz; order:4) was applied on the raw timeserie. The signal was then downsampled at 1000Hz. The SC signal was decomposed in **phasic** and **tonic** components using the filtering method (cutoff of 0.05Hz). 

### Extracted features
SCR were detected from the **phasic** SC by finding the local maxima in the signal by comparison of neighboring values (see [scipy.signal.find_peaks](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html) with `default` parameters). Peaks were corrected using a minimum relative amplitude threshold. More specifically, the signal was first scaled using a max scaling approach. 

**SCR peaks** with an amplitude above 0.1 were kept, the other were discarded (see [neurokit2.signal.signal_findpeaks](https://neuropsychology.github.io/NeuroKit/functions/signal.html#signal-findpeaks); `relative_height_min=0.1`, `relative_max=True`).

**SCR onsets** are define as the local maxima on the amplitude-inverted SC timeserie that is the closest to the previously found peaks.

**SCR rec.t/2** or **SCR recovery** (half recovery) corresponds to the timepoint for which the SCR amplitude has declined to 50% of the SCR maximum amplitude (SCR maximum amplitude = amplitude peak - amplitude onset).

For a visualization of those features, please refer to the figure under the [`eda_process` section](https://neuropsychology.github.io/NeuroKit/functions/eda.html#neurokit2.eda.eda_process) in the Neurokit2 documentation or Fig. 2.15 in [Boucsein 2012](https://doi.org/10.1007/978-1-4614-1126-0) (p. 154).

### Features to extract for the analyses

The following features will be extracted for each window. A window is defined from the onset of one stimulus to the onset of the following stimulus.This decision was made to make sure to include SCRs that could arise at the end of a given stimulus since we are using dynamic stimuli.

**SCR rate**: Number of SCR in a given window / Length of the window.

**SCR amp. max**: Maximum SCR height in a given window.

**SCR amp. mean**: Mean of SCRs height in a given window.

**SCR amp. sd**: Standard deviation of SCRs height in a given window.

## Sources of artifacts

**Artifacts** are defined as "recorded biosignal which do not stem from the signal source in question." (Bousein et al., 2012).

Different sources of artifacts can influence the quality of the SC timeseries. Some of those sources are described below. Our filtering steps should already have eliminated some of them.

### MR-related artifacts

Since the SC is an electrical signal, acquiring it concurrently with fMRI data introduce high frequency artifacts (see [Bottenhorn et al., 2023](https://pmc.ncbi.nlm.nih.gov/articles/PMC12369973/)). However, given the low frequency of the signal of interest, our filtering procedure have mitigated some of these artifacts. However, some level of noise remains, but that should be tolerable and the threshold on the SCR detection algorithm should have not detect that noise as SCR. 

**How to detect**: MR-related noise should be constant throughout the run. If a SCR is detected but its shape has the same shape as the surrounding noise, it should be flagged. Or in general, if the detected SCR just doesn’t have the expected shape, it should be flagged.

**How to correct**: If any falsely detected SCR were identified based on the criterion above, remove the SCRs. If there is any doubt, please flag the segment and add it in the GSlide.  

### Power line noise

Those artifacts should not be present in our preprocessed signal since we applied a lowpass filter at 3Hz (Power line at 60Hz). 

### Motion

Motion can affect the quality of the signal acquired, making it unusable. Motion can also introduce non-specific SCRs (NS.SCRs). 

**How to detect**: motion should be fairly easy to identify. It could look like high frequency jitter, sharp spikes, vertical jumps (very steep slope). Since SCR is a relatively slow response, any sudden change / fluctuation should be flagged as noise.

**How to correct**: remove any SCRs detected in a segment containing motion artifacts. Write down the onset and duration of that segment.


### Respiratory artifacts

Deep inhalation and breath holding can introduce NS.SCRs. Deep breath can also influence the latency of SCR by "delaying" the onset. In our study, we could expect SCR and different breathing patterns to covary (orienting- or defensive-response). 

**How to correct**: it is difficult to dissociate the respiratory response from the SCR one. The only thing that we will do a posteriori is to provide the onset and duration of segments containing deep breaths. This will be retrieved automatically from the respiratory waveforms.

---

## Import dependencies

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import neurokit2 as nk
import matplotlib.pyplot as plt
from bids.layout import BIDSLayout
from peakdet import Physio, operations, save_physio, load_physio

## Instantiate path

Since our data are organized in BIDS format, we can use `pybids` querying functionalities.

---

To learn more about [pybids](https://bids-standard.github.io/pybids/).
<br>To learn more about the BIDS format, check out the [BIDS paper](https://doi.org/10.1038/sdata.2016.44) and the [BIDS spec](https://bids-specification.readthedocs.io/en/stable/).

<div class="alert alert-block alert-danger">
<b>Change the path</b> in cell below to reflect the actual directory where the preprocessed data are stored.
</div>

In [ ]:
path = '/path/to/emotion-videos.physprep_eda'
layout = BIDSLayout(path, validate=False, is_derivative=True)

The BIDSLayout object can be used to return subject id that are found in `path`, or session and run id for a specific subject. Let's see how it works.

In [ ]:
subjects = layout.get_subject()
# Let's see the subject id that we have in our directory
print(subjects)

In [ ]:
# To get the session id...
sessions = layout.get_session()
print(sessions)

Here we get the session id across all participants. However, the number of sessions and session ids could change from one participant to another. If we want to check the sessions related to one specific subject, we can use the `subject` parameter.

In [ ]:
sessions_06 = layout.get_session(subject='06')
print(sessions_06)

We can do the same for the runs. To get either the run ids across all subjects, or for a specific subject and a specific session.

In [ ]:
# Across participants and sessions
runs = layout.get_run()
print(runs)

# Within a specific participant across sessions
runs_06 = layout.get_run(subject='06')
print(runs_06)

# Within a specific participant for a specific session
runs_06_001 = layout.get_run(subjest='06', session='001')
print(runs_06_001)

## Load data

We are going to load two different files:

- The first file (content store in `preproc_physio`) contains the preprocessed timeseries (continuous data). 
    - Example: `sub-01_ses-001_task-emotion_run-01_desc-preproc_physio.tsv.gz`

- The second file (content store in `events_physio`) contains features that were extracted from the preprocessed timeseries (sparse data).
    - Example: `sub-01_ses-001_task-emotion_run-01_desc-physio_events.tsv`

The features are the following:
- **PPG**: systolic peak (var: `systolic_peak_corrected`)
- **ECG**: r peak (var: `r_peak_corrected`)
- **RSP**: index at the maximum inhalation amplitude (var: `inhale_max`); index at the maximum exhalation amplitude (var: `exhale_max`)
- **EDA**: indext at the maximum amplitude of the skin conductance response (var: `scr_peak`); onset of the skin conductance response (var: `scr_onset`)

In [2]:
# Defining the feature name related to each modality
feat_dict = {
    'PPG': {
        'feat': 'systolic_peak_corrected',
        'feat_corrected': 'systolic_peak_manually_corrected'
    },
    'ECG': {
        'feat': 'r_peak_corrected',
        'feat_corrected': 'r_peak_manually_corrected'
    },
    'RSP': {
        'feat': 'inhale_max',
        'feat_corrected': 'inhale_max_manually_corrected',
        'feat_trough': 'exhale_max',
        'feat_trough_corrected': 'exhale_max_manually_corrected'
    },
    'EDA': {
        'feat': 'scr_peak',
        'feat_corrected': 'scr_peak_manually_corrected',
        'feat_trough': 'scr_onset',
        'feat_trough_corrected': 'scr_onset_manually_corrected'
    }
}

In [ ]:
def load_preproc_data(layout, sub, ses, run, modality="EDA", sampling_rate=1000):
    """
    Parameters
    ----------
    sub: str
        subject id (e.g., '01').
    ses: str
        session id (e.g., '001').
    run: str
        run id (e.g, '01').
    modality: str
        physiological modality. Could either be "ECG", "PPG", "RSP" or "EDA".
        Default to "EDA".
    
    Returns
    -------
    physio: Physio
        Physio object containing the preprocessed timeseries and the extracted peaks (and troughs).
    """
    
    # Load physio events to retrieve the peaks and troughs
    ## The line below will retrieve all the sub-{sub}_ses-{ses}_task-xx_run-{run}_desc-physio_events.tsv file 
    ## For subject `sub`, session `ses` and run `run`
    events_physio= layout.get(subject=sub, session=ses, run=run, suffix='events')
    entities = events_physio[0].get_entities()
    events_physio = pd.read_csv(events_physio[0], sep='\t')
    
    # Load preprocessed timeseries
    ## The line below will retrieve all the sub-{sub}_ses-{ses}_task-xx_run-{run}_desc-preproc_physio.tsv.gz file 
    ## For subject `sub`, session `ses` and run `run`
    preproc_physio = layout.get(subject=sub, session=ses, run=run, suffix='physio', extension='tsv.gz')
    preproc_physio = pd.read_csv(preproc_physio[0], sep='\t')
    
    # Create Physio object
    physio = Physio(np.array(preproc_physio[f'{modality}_clean']), fs=sampling_rate)
    physio = operations.peakfind_physio(physio)
    
    # Retrieve features name based on `modality`
    feat = feat_dict[modality]['feat']
    feat_corrected = feat_dict[modality]['feat_corrected']
    
    physio._metadata['peaks'] = np.array(events_physio[events_physio['trial_type']==feat]['onset']*sampling_rate).astype(int)
    
    if modality in ["EDA", "RSP"]:
        feat_trough =  feat_dict[modality]['feat_trough']
        feat_trough_corrected = feat_dict[modality]['feat_trough_corrected']
        
        physio._metadata['troughs'] = np.array(events_physio[events_physio['trial_type']==feat_trough]['onset']*sampling_rate).astype(int)
    
    return physio

In [ ]:
sub = '01'
ses = '001'
run = '01'
modality = 'EDA'
sampling_rate = 1000

In [ ]:
physio = load_preproc_data(layout, sub, ses, run, modality=modality, sampling_rate=sampling_rate)

## Manual correction

For the manual QC, we will use [peakdet](https://peakdet.readthedocs.io/en/latest/index.html), more specifically the [`edit_physio` function](https://peakdet.readthedocs.io/en/latest/user_guide/editing.html).

### To remove some peaks

**Command**: Left click + drag
<br>**Behavior**: a red box will appear where you cliked + dragged your cursor. All the peaks in that box will be deleted

### To add some peaks

**Command**: Right click + drag
<br>**Behavior**: a green box will appear where you clicked + dragged your cursor. A peak will be added 
 at the maxima in that window. One peak is added per box, meaning that if you want to add multiple peaks, you'll need to work on smaller segments.
 
### To undo any operation

**Command**: ctrl+z or command+z on Mac
<br>**Behavior**: the operation you previously did will be undone.

### To move along your timeserie

**Command**: click on the button with the 4 arrows at the top of your window. Click + move your cursor while maintaining the left click.

### To zoom in and out

**Command - Zoom in**: click on the button with a little magnifying glass. Draw a rectangle on the segment you want to zoom in.
<br>**Command - Zoom out**: click on the button with the little house at the top of your window.

### To close the interactive viewer

**Command**: ctrl+q or command+q on Mac
<br>**Behavior**: the interactive viewer will close, but all the changes will be stored automatically in the history.

### Physio object history

**Command**: `<physio object>.history`
<br>**Behavior**: all operations performed on a physio object will be stored in its history, for example indexes of removed/added peaks.

In [ ]:
%matplotlib qt
physio = operations.edit_physio(physio)

In [ ]:
# The history shows the index related to added/deleted peaks
print(physio.history)